In [1]:
import timm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# os.environ['KAGGLE_CONFIG_DIR'] = "/content"

In [ ]:
# %cd /content/drive/MyDrive/
# !kaggle datasets download -d ashery/chexpert -p chexpert_data

In [11]:
model = timm.create_model('vit_base_patch16_224', pretrained=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [18]:
class CheXpertDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.labels_df = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        img_name = self.labels_df.iloc[idx]['Path']
        img_path = os.path.join(self.img_dir, os.path.basename(img_name))

        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)


        labels = self.labels_df.iloc[idx][['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural Effusion']].fillna(0).values.astype('float32')

        return image, labels

In [19]:
transform = transforms.Compose([
    transforms.Resize((244, 244)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

In [20]:
train_dataset = CheXpertDataset(
    csv_file='/content/drive/MyDrive/chexpert_data_v2/train.csv',
    img_dir='/content/drive/MyDrive/chexpert_data_v2_resized/train',
    transform=transform
)

In [21]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [22]:
val_dataset = CheXpertDataset(
    csv_file='/content/drive/MyDrive/chexpert_data_v2/valid.csv',
    img_dir='/content/drive/MyDrive/chexpert_data_v2_resized/valid',
    transform=transform
)

In [23]:
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [24]:
# !git commit -m "Update LLRF notebook: added custom dataset loader and ViT fine-tuning code"